In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pymysql

db = pymysql.connect(
    host="localhost",
    user="root",
    password="mahadev@9096",
    database="customer_behavior",
    port=3306
)

cur = db.cursor()

 # Q1. What is the total revenue generated by male vs. female customers?

In [2]:
query = """ select gender, SUM(purchase_amount) as revenue
from customer_behavior.customers
group by gender """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data)
df.head()

,0,1
0,Male,157890
1,Female,75191


 # Q2. Which customers used a discount but still spent more than the average purchase amount? 

In [4]:
query = """ SELECT customer_id, purchase_amount
FROM customer_behavior.customers
WHERE discount_applied = 'Yes'
  AND purchase_amount >= (
        SELECT AVG(purchase_amount)
        FROM customer_behavior.customers
     );
 """

cur.execute(query)

data = cur.fetchall()
df = pd.DataFrame(data)
df.head()

,0,1
0,2,64
1,3,73
2,4,90
3,7,85
4,9,97


 # Q3. Which are the top 5 products with the highest average review rating?

In [9]:
query = """
SELECT 
    item_purchased,
    ROUND(AVG(review_rating), 2) AS average_product_rating
FROM customers
GROUP BY item_purchased
ORDER BY average_product_rating DESC
LIMIT 5;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns=["item_purchased", "average_product_rating"])

print(df)


  item_purchased  average_product_rating
0         Gloves                    3.86
1        Sandals                    3.84
2          Boots                    3.82
3            Hat                    3.80
4        Handbag                    3.78


Q4. Compare the average Purchase Amounts between Standard and Express Shipping. 

In [10]:
query = """
select shipping_type, 
ROUND(AVG(purchase_amount),2)
from customer_behavior.customers
where shipping_type in ('Standard','Express')
group by shipping_type;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data)

print(df)

          0      1
0   Express  60.48
1  Standard  58.46


Q5. Do subscribed customers spend more? Compare average spend and total revenue 
--between subscribers and non-subscribers.

In [13]:
query = """
SELECT 
    subscription_status,
    COUNT(customer_id) AS total_customers,
    ROUND(AVG(purchase_amount),2) AS avg_spend,
    ROUND(SUM(purchase_amount),2) AS total_revenue
FROM customers
GROUP BY subscription_status
ORDER BY total_revenue DESC, avg_spend DESC;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data, columns=[
    "subscription_status",
    "total_customers",
    "avg_spend",
    "total_revenue"
])

print(df)


  subscription_status  total_customers avg_spend total_revenue
0                  No             2847     59.87        170436
1                 Yes             1053     59.49         62645


Q6. Which 5 products have the highest percentage of purchases with discounts applied?

In [15]:
query = """
SELECT item_purchased,
       ROUND(100.0 * SUM(CASE WHEN discount_applied = 'Yes' THEN 1 ELSE 0 END)/COUNT(*),2) AS discount_rate
FROM customer_behavior.customers
GROUP BY item_purchased
ORDER BY discount_rate DESC
LIMIT 5;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data)

print(df)

          0      1
0       Hat  50.00
1  Sneakers  49.66
2      Coat  49.07
3   Sweater  48.17
4     Pants  47.37


Q7. Segment customers into New, Returning, and Loyal based on their total 
-- number of previous purchases, and show the count of each segment. 

In [17]:
query = """
with customer_type as (
SELECT customer_id, previous_purchases,
CASE 
    WHEN previous_purchases = 1 THEN 'New'
    WHEN previous_purchases BETWEEN 2 AND 10 THEN 'Returning'
    ELSE 'Loyal'
    END AS customer_segment
FROM customer_behavior.customers)

select customer_segment,count(*) AS "Number of Customers" 
from customer_type 
group by customer_segment;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data)

print(df)

           0     1
0      Loyal  3116
1  Returning   701
2        New    83


Q8. What are the top 3 most purchased products within each category? 

In [18]:
query = """
WITH item_counts AS (
    SELECT category,
           item_purchased,
           COUNT(customer_id) AS total_orders,
           ROW_NUMBER() OVER (PARTITION BY category ORDER BY COUNT(customer_id) DESC) AS item_rank
    FROM customer_behavior.customers
    GROUP BY category, item_purchased
)
SELECT item_rank,category, item_purchased, total_orders
FROM item_counts
WHERE item_rank <=3;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data)

print(df)

    0            1           2    3
0   1  Accessories     Jewelry  171
1   2  Accessories  Sunglasses  161
2   3  Accessories        Belt  161
3   1     Clothing      Blouse  171
4   2     Clothing       Pants  171
5   3     Clothing       Shirt  169
6   1     Footwear     Sandals  160
7   2     Footwear       Shoes  150
8   3     Footwear    Sneakers  145
9   1    Outerwear      Jacket  163
10  2    Outerwear        Coat  161


Q9. Are customers who are repeat buyers (more than 5 previous purchases) also likely to subscribe?

In [19]:
query = """
SELECT subscription_status,
       COUNT(customer_id) AS repeat_buyers
FROM customer_behavior.customers
WHERE previous_purchases > 5
GROUP BY subscription_status;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data)

print(df)

     0     1
0  Yes   958
1   No  2518


Q10. What is the revenue contribution of each age group? 

In [20]:
query = """
SELECT 
    age_group,
    SUM(purchase_amount) AS total_revenue
FROM customer_behavior.customers
GROUP BY age_group
ORDER BY total_revenue desc;
"""

cur.execute(query)
data = cur.fetchall()

df = pd.DataFrame(data)

print(df)

             0      1
0  Young Adult  62143
1  Middle_aged  59197
2        Adult  55978
3       Senior  55763
